In [1]:
import os

os.chdir("../")

In [2]:
%pwd

'c:\\Users\\admin\\Desktop\\Project\\e2e\\ml_practice'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str



In [4]:
from mlProject.constants import *
from mlProject.utils.comman import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath  = CONFIG_FILE_PATH, params_filepath =  PARAMS_FILE_PATH, schema_filepath =  SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params =  read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path = config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name,
        )

        return model_trainer_config

In [8]:
import os
import pandas as pd
from sklearn.linear_model import ElasticNet
from mlProject.utils import logger
import joblib


In [11]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config =  config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_X =  train_data.drop([self.config.target_column],axis=1)
        test_X =  test_data.drop([self.config.target_column],axis=1)

        train_Y =  train_data[[self.config.target_column]]
        test_Y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=12)
        lr.fit(train_X,train_Y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
        

In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer  = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-07-30 13:36:22,425: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2023-07-30 13:36:22,431: INFO: comman: yaml file: params.yaml loaded successfully]
[2023-07-30 13:36:22,440: INFO: comman: yaml file: schema.yaml loaded successfully]
[2023-07-30 13:36:22,442: INFO: comman: created directory at: artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'root_dir'"